<a href="https://colab.research.google.com/github/mrrajatgarg/kaggle/blob/master/DogVsCatKerasCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras

In [5]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mrrajatgarg","key":"ae5a419aa72b781b3ca5459616930b23"}'}

In [6]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Jan 26 07:52 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
# List available datasets.
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
mohansacharya/graduate-admissions                             Graduate Admissions                                   9KB  2018-12-28 10:07:14           5045  
karangadiya/fifa19                                            FIFA 19 complete player dataset                       2MB  2018-12-21 03:52:59           4176  
lava18/google-play-store-apps                                 Google Play Store Apps                                2MB  2018-09-18 20:49:49          32942  
fivethirtyeight/fivethirtyeight-comic-characters-dataset      FiveThirtyEight Comic Characters Dataset            577KB  2019-01-13 15:02:19            639  
gsutters/the-human-freedom-index                    

In [11]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 71.9MB/s]
 98% 265M/271M [00:04<00:00, 72.6MB/s]
100% 271M/271M [00:04<00:00, 57.1MB/s]
 99% 537M/543M [00:09<00:00, 58.6MB/s]
100% 543M/543M [00:09<00:00, 60.0MB/s]


In [0]:
import os
from tqdm import tqdm
import cv2

In [13]:
!ls

drive	kaggle.json  sampleSubmission.csv  train.zip
kaggle	sample_data  test1.zip


In [0]:
TRAIN_DIR = './train'
IMG_SIZE = 50

In [0]:
os.mkdir('training_data_data')

In [16]:
!ls

drive	kaggle.json  sampleSubmission.csv  training_data_data
kaggle	sample_data  test1.zip		   train.zip


In [0]:
#os.mkdir('training_data_data')
os.mkdir('training_data_data/cats')
os.mkdir('training_data_data/dogs')

In [20]:
!ls

drive	kaggle.json  sampleSubmission.csv  training_data_data
kaggle	sample_data  test1.zip		   train.zip


In [48]:
from zipfile import ZipFile

file_name = "train.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

Extracting the files...
Done


In [0]:
def create_train_test_dir() :
    counter = 1
    for img in tqdm(os.listdir(TRAIN_DIR)) :
        word_label = img.split('.')[-3]
        if word_label=='cat':
            label = "cats"
        elif word_label == 'dog':
            label = "dogs"
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        cv2.imwrite("./training_data_data/"+label+'/'+str(counter)+'.jpg', img)
        #print("image saved"+str(counter))
        counter = counter+1

In [24]:
create_train_test_dir()

100%|██████████| 25000/25000 [00:28<00:00, 863.97it/s]


In [25]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(50,50,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

In [31]:
training_set = train_datagen.flow_from_directory('training_data_data', target_size=(50,50), batch_size=32, class_mode='binary')

Found 25000 images belonging to 2 classes.


In [0]:
from PIL import Image

In [33]:
model.fit_generator(training_set, steps_per_epoch=8000, epochs=25, validation_steps=2000)

Epoch 1/25
8000/8000 [==============================] - 334s 42ms/step - loss: 0.4759 - acc: 0.7663
Epoch 2/25
8000/8000 [==============================] - 331s 41ms/step - loss: 0.3354 - acc: 0.8511
Epoch 3/25
8000/8000 [==============================] - 325s 41ms/step - loss: 0.2621 - acc: 0.8885
Epoch 4/25
8000/8000 [==============================] - 324s 40ms/step - loss: 0.2114 - acc: 0.9124
Epoch 5/25
8000/8000 [==============================] - 321s 40ms/step - loss: 0.1748 - acc: 0.9291
Epoch 6/25
8000/8000 [==============================] - 321s 40ms/step - loss: 0.1492 - acc: 0.9410
Epoch 7/25
8000/8000 [==============================] - 320s 40ms/step - loss: 0.1296 - acc: 0.9491
Epoch 8/25
8000/8000 [==============================] - 318s 40ms/step - loss: 0.1159 - acc: 0.9552
Epoch 9/25
8000/8000 [==============================] - 318s 40ms/step - loss: 0.1061 - acc: 0.9600
Epoch 10/25
8000/8000 [==============================] - 318s 40ms/step - loss: 0.0963 - acc: 0.9636

In [34]:
!ls

drive	kaggle.json  sampleSubmission.csv  train	       train.zip
kaggle	sample_data  test1.zip		   training_data_data


In [49]:
file_name = "test1.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

Extracting the files...
Done


In [36]:
!ls

drive	kaggle.json  sampleSubmission.csv  test1.zip  training_data_data
kaggle	sample_data  test1		   train      train.zip


In [55]:
img = cv2.imread('test1/10.jpg')
img = cv2.resize(img, (50,50))
img = img.reshape(1,50,50,3)
print(model.predict(img))

[[0.]]
